# [Module 5] Make Realtime Endpoint and Inference

이 노트북은 아래와 같은 작업을 합니다.
- Endpoint Config 생성
- RealTime Endpoint 생성
- Predictor 생성
- 추론 실행

In [1]:
import boto3

import pandas as pd
import sagemaker

from time import strftime, gmtime

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [2]:
%store -r # Load varibles stored

no stored variable or alias #
no stored variable or alias Load
no stored variable or alias varibles
no stored variable or alias stored


## EndpointConfig 생성
- Realtime Endpoint를 만들기 위한 설정 파일을 만듦니다.
    - model_name은 기존에 만든 model을 사용합니다. 위의 %store -r을 통해서 해당 이름을 가져 옵니다.

In [3]:
client = boto3.client("sagemaker")
endpoint_config_name = "churn-inference-pipeline-" + timestamp_prefix
response = client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants = [
        {
            'VariantName' : "AllTraffic",
            'ModelName' : model_name,
            'InitialInstanceCount' : 1,
            'InstanceType' : 'ml.m4.xlarge'
        }
    ]
)

NameError: name 'model_name' is not defined

## Endpoint 생성

In [ ]:
churn_endpoint_name = 'churn-inference-pipeline-endpoint-' + timestamp_prefix
churn_endpoint = client.create_endpoint(
    EndpointName = churn_endpoint_name,
    EndpointConfigName = endpoint_config_name
)

**EndpointStatus 가 InService**가 될때까지 기다려 주세요. 그렇지 않으면 아래 코드가 에러나 납니다.

In [ ]:
client.describe_endpoint(EndpointName = churn_endpoint_name)

### Create Predictor
- 위에서 생성한 Endpoint에 CSV 형태의 입력 데이타를 받을 수 있게 csv_serializer를 지정 함

In [ ]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
import sagemaker
sagemaker_session = sagemaker.Session()

predictor = RealTimePredictor(
    endpoint = churn_endpoint_name,
    sagemaker_session = sagemaker_session,
    serializer = csv_serializer,
    content_type = CONTENT_TYPE_CSV,
    accept = CONTENT_TYPE_JSON
)


### 추론 위한 입력 포맷을 만듦
- 이런 형태로 입력이 됨 (에:)
    - ' KS,186,510,400-6454,no,no,0,137.8,97,23.43,187.7,118,15.95,146.4,85,6.59,8.7,6,2.35,1' 

In [ ]:

def make_inference_format(sample):
    instance = str()
    for i, token in enumerate(sample):
        # print(token)
        if i > 0:
            instance = instance  + ',' + str(token) 
        else:
            instance = instance  +  str(token) 
    return instance



### Test 데이타에서 10명을 추론
- 10명 중에서 7번째 고객이 이탈이 가능하다고 결과를 냄. 

In [ ]:
test_df = pd.read_csv("churn_data/batch_transform_test.csv", header=None)

for i in range(10):
    sample = test_df.iloc[i,:]
    instance = make_inference_format(sample)
    print("instance: \n", instance)

    payload = instance
    churn_result = predictor.predict(payload)
    print("Churn result?: \n", churn_result)
    print("")

## Delete Endpoint
- 작업을 완료 했으면, 필요 없는 리소스는 삭제

In [ ]:
client.delete_endpoint(EndpointName=churn_endpoint_name)